### Distribution Transformation

Distribution transformation is a very useful tool which will be extensively used with the copula concept that we discuss in the next Section.
The technique transforms every random variables from uniform to any distribution and vice versa and is called \emph{probability integral transform} or (percentile-to-percentile transform). 

Computationally, this method involves computing the quantile function of the distribution, in other words, computing the cumulative distribution function (CDF) of the distribution (which maps a number in the domain to a probability between 0 and 1) and then inverting that function. We won't go into the details but we will just show few examples of how this can be done in $\tt{python}$.

For example, imagine that $\mathbb{P}(X)$ is the standard normal distribution with mean zero and standard deviation one. If we want to convert uniformly distributed samples to standard normal we need to apply the inverse of CDF to each sample. Below few examples which exploit $\tt{scipy.stats}$ which defines many useful statistical distributions.

(Remember that the Uniform samples have to be interpreted as cumulative probabilities)

In [1]:
from scipy.stats import uniform, norm

uniform_samples = [0.5, .975, 0.995, 0.999999]

print ("Unif.\t\tStd. Normal")
for u in uniform_samples:
    print ("{:.7f}\t{:.8f}".format(u, norm.ppf(u)))


Unif.		Std. Normal
0.5000000	0.00000000
0.9750000	1.95996398
0.9950000	2.57582930
0.9999990	4.75342431


The same transformation may be applied directly on the entire sample.

In [18]:
from scipy import stats

x = stats.uniform(0, 1).rvs(10000)

Next we apply $\tt{ppf()}$ to the list of $x$ directly.

In [6]:
norm = stats.norm() 
x_trans = norm.ppf(x)

<img src="uniform_gauss.png">

If we plot them togheter in a 2D plot we can get a sense of what is going on when using the inverse CDF transformation:

<img src="lesson6_files/lesson6_7_0.png">

The inverse CDF stretches the outer regions of the uniform to yield a normal distribution. 

The nice thing of the technique is that it can be used with any arbitrary (univariate) probability distributions, like for example [t-Student](https://en.wikipedia.org/wiki/Student%27s_t-distribution) or [Gumbel](https://en.wikipedia.org/wiki/Gumbel_distribution)

In [45]:
tstudent = stats.distributions.t(4)
x_trans = tstudent.ppf(x)

<img src="lesson6_files/lesson6_9_0.png">

Clearly to do the opposite transformation from an arbitray distribution to the uniform(0, 1) we can just apply the inverse of the inverse CDF, which is the CDF itself...

In [46]:
x = stats.uniform(0, 1).rvs(10000)
norm = stats.norm() 
x_trans = norm.ppf(x)
x_uniform = norm.cdf(x_trans)

<img src="lesson6_files/lesson6_11_0.png">

## Copula

In probability theory a *copula* $C(U_1, U_2, \ldots, U_n, \rho)$ is a multivariate (multidimensional) cumulative distribution function for which the marginal probability distribution (the probability distribution of each dimension) of each variable is uniform on the interval $[0, 1]$ ($U_i \approx$~Uniform(0,1)). $\rho$ represent the correlation between each variable. 

*Sklar's theorem* states that any multivariate joint distribution can be written in terms of univariate marginal distribution functions and a copula which describes the dependence structure between the variables.

Copulas are used to describe the dependence between random variables and have been used widely in quantitative finance to model risk.

Despite the obscure and daunting definition the concept of copula is quite simple so let's try to clarify it a bit with a practical example.

### Example Problem Case
Imagine to measure two variables that are correlated. For example, we look at various rivers and for every river we look at its maximum water level, and also count how many months each river caused flooding. 

For the probability distribution of the maximum level of the river we know that maximums are Gumbel distributed, while the number of flooding can be modelled according to a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution).

Clearly it is pretty reasonable to assume that the maximum level and the number of floodings is going to be correlated, however we don't know how we could model that correlated probability distribution. Above we only specified the distributions for individual variables, irrespective of the other one (i.e. the marginals), in reality we are dealing with the joint distribution of both of these together. 

And here is where copulas come to our rescue.

Copulas essentially allow to decompose a joint probability distribution into their marginals (which by definition have no correlation) and a function which couples (hence the name) them together and thus allows us to specify the correlation separately. The copula is that coupling function.

## Adding Correlation with Gaussian Copulas
We are actually almost done indeed, we saw before how to convert anything uniformly distributed to an arbitrary probability distribution. So that means we need to generate uniformly distributed data with the correlation we want and then transform the marginals into the desired distributions. 

How do we do that ? 

* simulate from a multivariarte Gaussian with the specific corrrelation structure (since we know very well how to do it);
* transform each Gaussian marginal to uniform;
* finally transform the uniforms to whatever we like.

So let's sample from a multivariate normal (2D) with a 0.5 correlation.

In [47]:
mvnorm = stats.multivariate_normal(mean=[0, 0] , cov=[[1, 0.5],
                                                      [0.5, 1]])
x = mvnorm.rvs(100000)

<img src="lesson6_files/lesson6_14_0.png">

Now use what we have just seen to tranform the marginals to uniform using the $\tt{cdf}$ function of the normal distribution ($x$ is a 2D vector in this case, but tranformation will be applied separately on each component)

In [48]:
norm = stats.norm()
x_unif = norm.cdf(x)

<table>
    <tr>
        <td><img src="lesson6_files/lesson6_16_0.png" width=300></td>
        <td><img src="copula_3d.png" width=300></td>
    </tr>
</table>

These plots above is usually how copulas are visualized.

Finally we can just transform the marginals again from uniform to what we want (i.e. Gumbel and Beta in our river example): 

In [49]:
m1 = stats.gumbel_l()
m2 = stats.beta(a=10, b=3)

# transform U1 into Gumbel
# [:, 0] means all entries of dim 0  
x1_trans = m1.ppf(x_unif[:, 0])

#transform U2 into Beta
# [:, 1] means all entries of dim 1
x2_trans = m2.ppf(x_unif[:, 1])

To see that it is actually working as expected we should now compare our scatter plot with correlation to the joint distribution of the same marginals without correlation:

In [50]:
# sample from Gumbel
x1 = m1.rvs(10000)
# sample from Beta
x2 = m2.rvs(10000)

<table>
    <tr>
        <td><img src="lesson6_files/lesson6_18_0.png" width=300></td>
        <td><img src="lesson6_files/lesson6_20_0.png" width=300></td>
    </tr>
</table>

Using the uniform distribution as a common base for our transformations we can easily introduce correlations and flexibly construct complex probability distributions. Clearly this is directly extendeable to higher dimensional distributions as well.

### Generate Correlated Distributions
Let's now see how copulas can be used to generate numbers from correlated distributions. These are the steps to follow:

* generate a random vector $\mathbf{x}=(x_1, x_2,\ldots)$ from a multivariate distribution with the desired correlation;
* determine the single $U_i(x_i)$ by applying $\tt{cdf}$ to each $x_i$;
* transform again each $U_i(x_i)$ to the desired marginal distributions using $\tt{ppf}$.

Each component of the vector $\mathbf{x}$ is now transformed as it was drawn from the desired marginals with the appropriate correlation.

A practical application concerns the probability of default. Imagine there are three companies (A, B and C) which have a cumulative probability of defaulting within the next two years of 10%.

Let’s try to compute the probabilities to have the three of them all defaulting within the next two years in the cases with independent and correlated default probabilities.

In the first case (independent probabilities), the odds to get three defaults within two years is the product of the single probabilities, hence:
$$\mathbb{P}_{\mathrm{uncorr}}= 10\%\cdot 10\%\cdot 10\% = 0.1\%$$

We can verify this in $\tt{python}$ by applying the Monte Carlo algorithm outlined above: generate a random sample from an uncorrelated multivariate normal distribution, then transform each sample
( $[ x_A , x_B , x_C ]$ ) into the uniform distribution with the $\tt{norm.cdf}$ function (i.e. we convert the samples into probabilities) and then count how many times the three of them are lower than 10%. The final probability will be the ratio of the numer of successes by the total trials.

In [58]:
from scipy.stats import multivariate_normal, uniform, norm
import numpy

numpy.random.seed(10)
trials = 500000
mvnorm_no_corr = multivariate_normal(mean=[0, 0, 0], cov=[[1, 0, 0],
                                                          [0, 1, 0],
                                                          [0, 0, 1]])

defaults = 0
x = mvnorm_no_corr.rvs(trials)
x_trans = norm.cdf(x)
for i in range(len(x_trans)):
    if x_trans[i][0] < 0.1 and x_trans[i][1] < 0.1 and \
       x_trans[i][2] < 0.1:
        defaults += 1

print ("Defaults w/o correlation: {:.2f}%".format(defaults/trials*100))

Defaults w/o correlation: 0.10%


If we repeat the same Monte Carlo experiment with perfectly correlated default probabilities
we have

In [59]:
from scipy.stats import multivariate_normal, uniform, norm
import numpy

numpy.random.seed(10)
trials = 500000
mvnorm_no_corr = multivariate_normal(mean=[0, 0, 0], 
                                     cov=[[1, 0.99999, 0.99999],
                                          [0.99999, 1, 0.99999],
                                          [0.99999, 0.99999, 1]])

defaults = 0
x = mvnorm_no_corr.rvs(trials)
x_trans = norm.cdf(x)
for i in range(len(x_trans)):
    if x_trans[i][0] < 0.1 and x_trans[i][1] < 0.1 and \
       x_trans[i][2] < 0.1:
        defaults += 1

print ("Defaults w/o correlation: {:.2f}%".format(defaults/trials*100))

Defaults w/o correlation: 9.96%


In this case the result is 10%, like we had only one single company, indeed being perfectly
correlated either there is no default or three "simultaneous" defaults with 10% probability.

### Valuation of CDO
Suppose that the payment date on a CDO tranche are at times $\tau_i$. Define $\mathbb{E}_j$ the expected 
tranche principal at time $\tau$ and $D(\tau)$ the discount factor at time $\tau$. Suppose also that the spread
on a particular tranche (i.e. the number of basis point paid for protection on the remaining tranche principal) is $s$. 

The present value of the expected regular spread payments on the CDO is given by
\begin{equation}
s\cdot A = s\cdot \sum_{j=1}^{m}(\tau_j - \tau_{j-1})\mathbb{E}_{j}D(\tau_j)
\label{eq:A}
\end{equation}
The expected loss between times $\tau_{j-1}$ and $\tau_j$ is $\mathbb{E}_{j-1}-\mathbb{E}_j$. For simplicity assume
the loss occurs only at the midpoint of the time interval, so the present value of the expected payoffs on the CDO tranche is
\begin{equation}
C=\sum_{j=1}^{m}(\mathbb{E}_{j-1}-\mathbb{E}_j)D\left(\frac{\tau_{j-1}+\tau_j}{2}\right)
\label{eq:C}
\end{equation}
The accrual payment due on the losses is finally given by
\begin{equation}
s\cdot B = s\cdot\sum_{j=1}^{m}\frac{1}{2}(\tau_j - \tau_{j-1})(\mathbb{E}_{j-1}-\mathbb{E}_j)D(\frac{\tau_{j-1}+\tau_j}{2})
\label{eq:B}
\end{equation}

The value of the tranche, valued from the point of view of the protection buyer is $C-sA-sB$. The breakeven spread 
on the tranche occurs when the present value of the payments equals the present value of the payoffs so

$$ s = \cfrac{C}{A+B}$$

Suppose that the tranche under consideration covers losses on the portfolio between $\alpha_L$ and $\alpha_H$ and define

$$n_L = \cfrac{\alpha_L n}{1-R}\qquad \mathrm{and}\qquad n_H = \cfrac{\alpha_H n}{1-R}$$
where $R$ is the recovery rate. Finally define $m(x)$ as the smallest integer greater than $x$.
By definition the tranche principal stays $N$ while the number of defaults $k$ is less than $m(n_L)$, it is zero when the number of default is greater or equal to $m(n_H)$, otherwise is

$$\cfrac{\alpha_H -k(1-R)/n}{\alpha_H - \alpha_L}$$

The expected tranche principal at time $\tau_j$ conditional of the value of the factor $M$ is
$$\begin{equation}
\mathbb{E}_j(M) = \sum_{k=0}^{m(n_L)-1}\mathbb{P}(k, \tau_j|M) + \sum_{k=m(n_L)}^{m(n_H)-1}\mathbb{P}(k, \tau_j|M) \cfrac{\alpha_H -k(1-R)/n}{\alpha_H - \alpha_L}
\label{eq:E}
\end{equation}
$$

To compute the breakeven spread it is necessary to substitute Eq.~\ref{eq:E} into Eq.~\ref{eq:A},~\ref{eq:B} and~\ref{eq:C}
and we need to integrate the result over the variable $M$ (remember that has a standard normal distribution). 
The integration is quite complicated and is best accomplished with a technique called *Gaussian quadrature* which exploits the approximation

$$\int_{-\infty}^{\infty}{\cfrac{1}{\sqrt{2}}e^{-M^{2}/2}g(M)dM} \approx \sum_{k=1}^{k=L}w_k g(F_k)$$
as $L$ increases, accuracy increases.